In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My\ Drive/Colab\ Notebooks/Signal-Phase-Busan
!pwd
!pip install dplython

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/Signal-Phase-Busan
/content/drive/My Drive/Colab Notebooks/Signal-Phase-Busan


In [ ]:
!python TOD.py --ixr-nm 동래교차로 --input-dir ./data --output-dir ./result --recommend F --max-tod 4

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(

In [ ]:
# pip install dplython

     |████████████████████████████████| 716kB 6.0MB/s 
  Created wheel for dplython: filename=dplython-0.0.7-cp36-none-any.whl size=747244 sha256=88f01f63f07e3e362f5f194956fd590704cf9b858858bbf9c75d5da8261d7f8d
  Stored in directory: /root/.cache/pip/wheels/0b/c5/dc/06ed265021b09111a8e7bd7f7577656a64e1293c10df09e0f5
Successfully built dplython





ㄴㅇㄴ
ㅇㄴㅇ






ㄴㅇ


ㄴㅁㅇ








ㄴㅇㄴㅇ

In [ ]:
--ixr-nm 동래교차로 --input-dir ./data --output-dir ./result --recommend F --max-tod 4

SyntaxError: ignored

In [ ]:
'''
Using dataset from smart intersection, time table with TOD labels is estimated by K-Means method
* Unit: 30 minute
* Single intersection
* Go-direction traffic includes right-turn traffic

* Input dataset:
- IMGN_DTCT_RAW_CLCT
- CMRA_MNGM

* Output:
- TOD table
- Traffic analysis according to each TOD period (Traffic: veh/30mins)

* Example code: python TOD.py --ixr-nm 내성교차로 --input-dir ./data --output-dir ./result --recommend F --max-tod 4

* Range of SC Interpretation
0.71-1.0 A strong structure has been found
0.51-0.70 A reasonable structure has been found
0.26-0.50 The structure is weak and could be artificial
< 0.25 No substantial structure has been found

'''
import os

import numpy as np
import pandas as pd
import statistics
import argparse
# from yellowbrick.cluster import KElbowVisualizer
from dplython import (DplyFrame, X, select, sift, count, head, arrange, mutate, group_by, summarize, inner_join)

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn import preprocessing
from collections import Counter

import matplotlib.pyplot as plt
import matplotlib.cm as cm

# import numpy as np
# from pandas import DataFrame
# import csv

In [ ]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--ixr-nm', required = True, help = 'name of intersection (IXR_NM)', type=str, default = '동래교차로')
# parser.add_argument('--input-dir', required = True, help = 'directory including inputs', type=str, default = './data')
# parser.add_argument('--output-dir', required = True, help = 'directory to save outputs', type=str, default = './result')
# parser.add_argument('--recommend', required = True, help = 'automatically choose the number of TOD(Y/N)', type=str, default = False)
# parser.add_argument('--max-tod', required = False, help = 'maximum number of TOD groups', type=int, default = 4)
# parser.add_argument('--cycle', required = False, help = 'length of cycle', type=int, default = 180)
# parser.add_argument('--rate-gap', required = False, help = 'threshold for flow rate difference on the same phase', type=float, default = 0.1)
# # parser.add_argument('--vis', required = False, help = 'visualize result(1) or not(0, default)', type=int, default = 0)
# args = parser.parse_args()

# pd.set_option('mode.use_inf_as_na', True)

# IXR_ID =""

In [ ]:
__ixr_nm = '동래교차로'
__input_dir = './data'
__output_dir = './result'
__recommend = False
__max_tod = 4
__cycle = 180
__rate_gap = 0.1

In [ ]:
def load_data():
    cctv_log = pd.read_csv(__input_dir + "/IMGN_DTCT_RAW_CLCT.csv")
    cctv_mst = pd.read_csv(__input_dir + "/CMRA_MNGM.csv")

    #cctv_mst = DplyFrame(cctv_mst) >> sift(X.IXR_NM  == IXR_NM) >> select(X.IXR_ID, X.CMRA_ID, X.ISTL_LCTN)
    cctv_mst = cctv_mst.loc[cctv_mst['IXR_NM']==__ixr_nm, ['IXR_ID', 'CMRA_ID', 'ISTL_LCTN']]
    global IXR_ID
    IXR_ID = cctv_mst.iloc[0].IXR_ID

    cctv_log['DATE'] = pd.DataFrame(pd.DatetimeIndex(cctv_log['CLCT_DT']).date)
    cctv_log['HOUR'] = pd.DataFrame(pd.DatetimeIndex(cctv_log['CLCT_DT']).hour)
    cctv_log['MINUTE'] = (pd.DataFrame(pd.DatetimeIndex(cctv_log['CLCT_DT']).minute) // 30) * 30
    cctv_log['temp_DAY'] = pd.to_datetime(cctv_log['DATE']).dt.dayofweek
    cctv_log.loc[cctv_log['temp_DAY'] < 5, 'DAY'] = int(0) #mon - fri
    cctv_log.loc[cctv_log['temp_DAY'] == 5, 'DAY'] = int(1) #sat
    cctv_log.loc[cctv_log['temp_DAY'] == 6, 'DAY'] = int(2) #sun
    df0 = DplyFrame(cctv_log) >> sift(X.IXR_ID  == IXR_ID) >> group_by(X.DATE, X.DAY, X.HOUR, X.MINUTE, X.CMRA_ID) >> summarize (GO_TRF = X.LRG_GO_TFVL.sum() + X.MDDL_GO_TFVL.sum() + X.SMAL_GO_TFVL.sum() + X.BIKE_GO_TFVL.sum() + X.LRG_RGHT_TFVL.sum() + X.MDDL_RGHT_TFVL.sum() + X.SMAL_RGHT_TFVL.sum() + X.BIKE_RGHT_TFVL.sum(),
                                                                                                                                 LEFT_TRF = X.LRG_LEFT_TFVL.sum() + X.MDDL_LEFT_TFVL.sum() + X.SMAL_LEFT_TFVL.sum() + X.BIKE_LEFT_TFVL.sum())

    # Extract records of selected crossroad
    df0 = pd.merge(df0, cctv_mst, how = "inner", on = "CMRA_ID")
    df0 = df0.sort_values(['DATE','HOUR','MINUTE','CMRA_ID']) # 소팅을 뭘로해? 데이, 아워, 미닛

    # Time frame from existing dataset
    tf = DplyFrame(df0.drop_duplicates(['DATE','DAY','HOUR','MINUTE'], keep='last')) >> select(X.DATE, X.DAY, X.HOUR, X.MINUTE)

    # Process the datastructure into pivot
    # cctv_list = sorted(cctv_mst['CMRA_ID'].unique())
    cctv_mst = cctv_mst.sort_values(['CMRA_ID'])
    df1 = tf.copy()

    for index, row in cctv_mst.iterrows():
        a = df0 >> sift(X.CMRA_ID == row['CMRA_ID']) >> select(X.DATE, X.DAY, X.HOUR, X.MINUTE, X.GO_TRF, X.LEFT_TRF)
        df1 = pd.merge(df1, a, how='left', on=['DATE', 'DAY', 'HOUR', 'MINUTE'], suffixes=('', '_' + str(row['ISTL_LCTN'])))

    df1 = df1.set_index(['DATE', 'DAY', 'HOUR', 'MINUTE'])
    df1 = df1.fillna(df1.rolling(window = 24, min_periods = 1, center = True).mean())
    df1 = df1.fillna(0)
    df1 = df1.reset_index()

    df1['TOTAL_TRF'] = DplyFrame(df1.iloc[:, 4:3 + len(cctv_mst) * 2].sum(axis=1, skipna=True))
    df1 = df1 >> sift(X.TOTAL_TRF > 0)
    # Name the cctv id and direction - for tod_traffic_analyzer

    #if cctv_list is a just list with cctv_id,
    # col_names = [CMRA + '_GO_RATE' for CMRA in cctv_list]
    # col_names.extend([CMRA + '_LEFT_RATE' for CMRA in cctv_list])
    # col_names = sorted(col_names)
    # col_names = ['TOD'] + col_names + ['TOTAL_TRF']
    # col_names

    dir_ = np.tile(np.array(['G','L']), len(cctv_mst))
    dir_index = np.tile(cctv_mst['CMRA_ID'], 2)
    dir_index = pd.DataFrame(dir_index, columns = ['CMRA_ID']).sort_values(['CMRA_ID']).reset_index(drop = True)
    dir_index['DIR'] = dir_
    dir_index = pd.merge(dir_index, cctv_mst[['CMRA_ID','ISTL_LCTN']], on = ['CMRA_ID'], how = 'left')

    #print(dir_index) #CMRA_ID, DIR(G/L), ISTL_LCTN
    # cols_g = cctv_list.copy()
    # cols_l = cctv_list.copy()
    # cols_g['CMRA_ID'] = cols_g['CMRA_ID'].astype(str) + 'GO_RATE'
    # cols_g['ISTL_LCTN'] = cols_g['ISTL_LCTN'].astype(str) + '직진비율'
    # cols_l['CMRA_ID'] = cols_l['CMRA_ID'].astype(str) + 'LEFT_RATE'
    # cols_l['ISTL_LCTN'] = cols_l['ISTL_LCTN'].astype(str) + '좌회전비율'
    #
    # cols = pd.concat([cols_g, cols_l]).sort_values(['CMRA_ID'])
    # cols = cols.ISTL_LCTN.values.tolist()
    # cols = ['TOD'] + cols + ['총교통량']

    return df1, dir_index

In [ ]:
def modefinder(df_tod):
    a = df_tod.groupby(['DAY', 'HOUR', 'MINUTE', 'TOD']).size().reset_index(name = 'counts')
    a = a.sort_values('counts', ascending = False).drop_duplicates(['DAY','HOUR','MINUTE'])

    return pd.DataFrame(a[['DAY', 'HOUR', 'MINUTE', 'TOD']])

In [ ]:
def estimate_tod(dat, a_rec, m_k):
    max_score = 0
    k = 1
    input_dat = data_normalize(dat)

    if a_rec == 'T':
        for i in range(2, m_k+1):
            
            model = KMeans(n_clusters = i, random_state = 10)
            TOD = model.fit_predict(input_dat)
            score = silhouette_score(input_dat, TOD)
            print("Silhouette score of {} clusters is {}".format(i, score))
            if(score > max_score):
                k = i
                max_score = score

        print("Best number of clusters is {} with score {}".format(k, max_score))
        model = KMeans(n_clusters=k, random_state=10)
        TOD = model.fit_predict(input_dat)
        dat['TOD'] = TOD

    elif a_rec == 'F':
        print('sd')
        model = KMeans(n_clusters=m_k, random_state=10)
        TOD = model.fit_predict(input_dat)
        score = silhouette_score(input_dat, TOD)
        print("Silhouette score of {} clusters is {}".format(m_k, score))

        dat['TOD'] = TOD

    dat_TOD = modefinder(dat)

    return dat_TOD

In [ ]:
def data_normalize(dat):
    temp = dat.drop(['DATE','DAY','HOUR','MINUTE'], axis = 1)
    a = temp.drop(['TOTAL_TRF'], axis = 1).to_numpy()
    b = temp['TOTAL_TRF'].to_numpy().reshape(-1,1)
    temp = pd.DataFrame(a/b)
    temp = temp.fillna(0)
    # temp = temp.dropna(axis = 0)

    #vertical normalize: total traffic across time-series
    minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1)).fit(dat[['TOTAL_TRF']])
    temp2 = pd.DataFrame(minmax_scale.transform(dat[['TOTAL_TRF']]))
    temp = pd.concat([temp, temp2], axis = 1)

    return temp

In [ ]:
def tod_traffic_analyzer(dat, d_i):
    print("dat")
    print(dat)

    a = dat.drop(['DATE','DAY','HOUR','MINUTE'], axis = 1).groupby('TOD').mean()
    a['TOTAL_TRF'] = a.drop(['TOTAL_TRF'], axis = 1).sum(axis = 1)

    a1 = a.drop(['TOTAL_TRF'], axis = 1).to_numpy()
    a2 = a['TOTAL_TRF'].to_numpy().reshape(-1,1)

    #calculate turning rate
    a1 = pd.DataFrame(a1/a2)
    turning_rate = pd.concat([round(a1,2), a['TOTAL_TRF']], axis = 1)
    turning_rate = turning_rate.reset_index()

    print("turning_rate")
    print(turning_rate)

    cols = d_i['ISTL_LCTN'].astype(str) + d_i['DIR'].astype(str)
    cols = cols.values.tolist()
    cols = ['TOD'] + cols + ['총교통량']
    turning_rate.columns = cols

    return turning_rate

In [ ]:
def phase_len(dat, d_i):
    co_sig_dir = pd.read_csv(__input_dir + "/CO_SIG_DIR.csv") #IXR_ID,PHASE,SIG_DIR1,SIG_DIR2
    mat_dir = pd.read_csv(__input_dir + "/MAT_DIR.csv") #CMRA_ID,DIR,SIG_DIR

    num_tod = len(dat)
    d_i = d_i.drop(['ISTL_LCTN'], axis = 1)

    dat_T = dat.drop(['TOD','총교통량'], axis = 1).T.reset_index()
    dat_T = dat_T.drop(['index'], axis = 1)
    turning_rate = pd.concat([d_i, dat_T], axis = 1)

    cols = ['CMRA_ID', 'DIR']
    for i in range(num_tod):
        cols.append('TOD' + str(i))
    turning_rate.columns = cols

    turning_rate = pd.merge(turning_rate, mat_dir, on = ["CMRA_ID","DIR"])

    sig_dir = co_sig_dir.loc[co_sig_dir['IXR_ID'] == IXR_ID].reset_index(drop=True)
    sig_cnt = sig_dir.stack().tolist()
    sig_cnt = pd.DataFrame.from_records(list(dict(Counter(sig_cnt)).items()), columns=['SIG_DIR','CNT'])

    sig_dir = pd.merge(sig_dir, sig_cnt, left_on = ['SIG_DIR1'], right_on = ['SIG_DIR'], how = 'left')
    sig_dir = pd.merge(sig_dir.drop(['SIG_DIR'], axis = 1), sig_cnt, left_on = ['SIG_DIR2'], right_on = ['SIG_DIR'], how = 'left')
    sig_dir = sig_dir.drop(['SIG_DIR'], axis = 1)
    sig_dir.columns = ['IXR_ID','PHASE','SIG_DIR1', 'SIG_DIR2', 'CNT1','CNT2']

    sig_result = pd.DataFrame()
    for i in range(num_tod):
        sig_rate = pd.merge(sig_dir, turning_rate[[cols[i+2], 'SIG_DIR']], left_on = ['SIG_DIR1'], right_on = ['SIG_DIR']) #for direction 1 in the same phase
        sig_rate = pd.merge(sig_rate.drop(['SIG_DIR'], axis = 1), turning_rate[[cols[i+2], 'SIG_DIR']], left_on = ['SIG_DIR2'], right_on = ['SIG_DIR']) #for direction 2 in the same phase
        sig_rate = sig_rate.drop(['IXR_ID', 'SIG_DIR'], axis = 1)
        sig_rate.columns = ['PHASE', 'SIG_DIR1', 'SIG_DIR2', 'CNT1','CNT2','RATE1','RATE2']

        sig_rate['RATE1'] = sig_rate['RATE1']/sig_rate['CNT1']
        sig_rate['RATE2'] = sig_rate['RATE2']/sig_rate['CNT2']
        sig_rate = sig_rate.drop(['CNT1','CNT2'], axis = 1)

        T_RATE = []
        for index, row in sig_rate.iterrows():
            if abs(row['RATE1']-row['RATE2']) > args.rate_gap:
                T_RATE.append((row['RATE1'] + row['RATE2'])/2) #나의 목적
            else:
                T_RATE.append(max(row['RATE1'], row['RATE2'])) #나의 목적

        sig_rate['T_RATE'] = pd.DataFrame(T_RATE)
        s_rate = sum(T_RATE)
        sig_rate['N_RATE'] = sig_rate['T_RATE']/s_rate
        sig_rate = sig_rate[['PHASE', 'N_RATE']]
        sig_rate['TOD'] = i
        sig_rate['SIG_LENGTH'] = round(sig_rate['N_RATE'] * args.cycle)

        sig_result = sig_result.append(sig_rate[['TOD', 'PHASE', 'SIG_LENGTH']])

    print(sig_result) #TOD, PHASE, SIGNAL_LENGTH
    return sig_result

In [ ]:
def visualize_traffic():
    return

In [ ]:
def run_process(dat, d_i, a_rec, max_k): #dat: grouped_raw, d_i: direction index, a_rec: automatic - recommandation, max_k: (maximum/prefer) k in K-means
    print('t0')
    tod_result = estimate_tod(dat, a_rec, max_k)
    print('t1')
    dat_tod = pd.merge(dat.drop(['TOD'], axis = 1), tod_result, left_on = ['DAY','HOUR','MINUTE'], right_on = ['DAY','HOUR','MINUTE'], how = 'left') #raw_group + TOD
    print('t2')
    info_result = tod_traffic_analyzer(dat_tod, d_i) #TOD, series of directions (rate), total traffic
    print('t3')
    sig_result = phase_len(info_result, d_i)

    tod_result['DAY'] = tod_result['DAY'].replace([0], '평일')
    tod_result['DAY'] = tod_result['DAY'].replace([1], '토요일')
    tod_result['DAY'] = tod_result['DAY'].replace([2], '일요일')

    return tod_result, info_result, sig_result

In [ ]:
if __name__ == '__main__':
    dirname = os.path.dirname(__output_dir)
    if os.path.dirname!='' and not os.path.exists(dirname):
        os.makedirs(dirname)

    if(max_k < 2):
        print("maximum number of TOD should be equal or larger than 2")
    else:
        df, dir_name = load_data()
        r1, r2, r3 = run_process(df, dir_name, __recommend, __max_tod) #r1: tod, r2: traffic attributes for each tod, r3: signal length

        r1.to_csv(__output_dir + '/tod_result_' + __ixr_nm + ".csv", index = False, encoding='utf-8-sig')
        r2.to_csv(__output_dir + '/traffic_characteristics_' + __ixr_nm + ".csv", index = False, encoding='utf-8-sig')
        r3.to_csv(__output_dir + '/signal_length_' + __ixr_nm + ".csv", index = False, encoding='utf-8-sig')

t0


KeyError: ignored